# Prototype table status generator 

Given a list of tables, apply various tests and produce a report in a table.


In [1]:

import BQhelper as bq
import pandas as pd

bq.project = "mlab-sandbox"

In [2]:
datasets=[
    'mlab-sandbox.base_tables', 'mlab-staging.base_tables', 'mlab-oti.base_tables',
    
]

TableQ="""SELECT * FROM `{dataset}.INFORMATION_SCHEMA.TABLES`"""

# r=bq.run_query(TableQ, dataSet=dataset)

def GetDataSetTables(dataset):
    print (dataset)
    r=bq.run_query(TableQ, dataset=dataset)
    r['dataset'] = dataset
    return pd.DataFrame(r, index=[dataset+'.'+t for t in r['table_name']])
# rr =GetDataSetTables(datasets[2])         

In [3]:
# scan datasets of  interest

# This should be a 3d structure rr[dataset][tabel][metadata col]
rr = pd.DataFrame()
for dataset in datasets:
    r = GetDataSetTables(dataset)
    rr = rr.append(r)
print (list(rr))

mlab-sandbox.base_tables
mlab-staging.base_tables
mlab-oti.base_tables
['table_catalog', 'table_schema', 'table_name', 'table_type', 'is_insertable_into', 'is_typed', 'creation_time', 'dataset']


In [4]:
# Indicate which tables are important
# We build this in early to make it possible to extend this to very large scales

effort = pd.Series([0 for i in range(len(rr.index))], index=rr.index, name='effort')
effort['mlab-oti.base_tables.ndt5'] = 9  # future flags, etc
rr['effort'] = effort
# print (rr)

In [9]:
ColumnQ = """
SELECT *
FROM {dataset}.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = {table}
"""

# r=bq.run_query(ColumnQ, dataset='mlab-oti.base_tables')

for trow in rr[rr['effort']>0].index:
    print (trow)
    r=bq.run_query(ColumnQ, dataset=trow['dataset'], table=trow['table_name'])
    print (r)

mlab-oti.base_tables.ndt5


TypeError: string indices must be integers

In [ ]:
print (r)


In [ ]:
SELECT *
FROM `mlab-sandbox.base_tables.INFORMATION_SCHEMA.TABLES`
JOIN ( SELECT TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, option_value FROM `mlab-sandbox.base_tables.INFORMATION_SCHEMA.TABLE_OPTIONS` WHERE option_name = 'description' )
USING ( TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME)